In [1]:
import pandas as pd
from datasets import Dataset
from transformers import DistilBertTokenizer

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

new_data_df = pd.read_clipboard()
new_data_df = new_data_df.sample(frac=1).reset_index(drop=True)
new_data_dataset = Dataset.from_pandas(new_data_df)
def preprocess(examples):
    return tokenizer(examples['text'], truncation=True, padding=True, max_length=512)
train_tokenized = new_data_dataset.map(preprocess, batched=True)

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

In [ ]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

model = DistilBertForSequenceClassification.from_pretrained('./model', num_labels=2)

training_args = TrainingArguments(
    output_dir="./model",
    overwrite_output_dir=True,
    evaluation_strategy="no",
    learning_rate=2e-5,
    per_device_train_batch_size=6,
    num_train_epochs=3,
    weight_decay=0.01,
    save_steps=1,
    save_total_limit=3,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
)
trainer.train()

In [5]:
import os
def save_checkpoint(model, checkpoint_dir):
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    model.save_pretrained(checkpoint_dir)

save_checkpoint(model, "./checkpoints")

model.save_pretrained('./model')
tokenizer.save_pretrained('./model')

('./model/tokenizer_config.json',
 './model/special_tokens_map.json',
 './model/vocab.txt',
 './model/added_tokens.json')